<a href="https://colab.research.google.com/github/cric96/DL-exercise/blob/main/test_with_rnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Recurrent Neural Network applied in Aggregate Computing
In this notebook, I tried to apply Recurrent Neural Networks (RNN) in the context of Aggregate Computing (AC).

## Model

Usually, RNN are trained in indipendent sequences. In case of Aggregate Computing, the temporal sequence are correlated with each other following a neighbourhood policy. 

The key idea here is:
- express the system as a graph. Offline we can imagine to access to the entire system
- in each time step, we aggregate data from the neighbours and then we use RNN to compute the right output

# Imports

In [2]:
import tensorflow as tf
import tensorboard
from datetime import datetime
from numpy.random import seed
%load_ext tensorboard

# Simple Graph

In this model, I model graph as:
$G = (N, E)$

Where, $N$ contains a feature vector and an output vector.

$E$ is expressed as adjacency matrix, so: $E_{i,j} = 1$ iff $i$ is neighbour of $j$.

Input is a matrix contains the concatenation of feature vector and output vector, something like: $ I(n) = <f(n), o(n)>$.
Where $<a, b, c>$ means the column-wise fector concatenation. 
For instance, giving a vector $a = [1, 2]$ and $b = [2, 3]$, $<a,b> = [1, 2, 2, 3]$.

The neighbour aggregation data can be computed leveraging matrix multiplication --- so increasing traning performance.
$$ reduction(E * F) $$
Where F contains all node features.
$reduction$ is a multiset function (e.g. summation, mean,...).

Pay attention, this model can be easly used in a decentralized situation.
Indeed, the data aggregation will be done without global adjency matrix but only retrieving neighbour data. The dense layer already works locally.

In [63]:
feature = tf.constant([
                     [1], 
                     [0],
                     [0],
                     [0],
                    ], dtype=tf.float32)
output = tf.constant([
                     [-1.0], 
                     [-1.0],
                     [-1.0],
                     [-1.0],
                    ], dtype=tf.float32)

input = tf.concat([feature, output], axis = 1)
input = input[:,tf.newaxis, :]
neigh = tf.constant([
                     [0, 1, 0, 0], 
                     [1, 0, 1, 0],
                     [0, 1, 0, 1],
                     [0, 0, 1, 0],
                    ], dtype=tf.float32)

ground = tf.constant([
                     [1, 0], 
                     [0, 1],
                     [0, 2],
                     [0, 3],
                    ], dtype=tf.float32)

# Forward Pass

Taking the output of previous step, adjecency matrix, and feature vector, this function compute the value of the next evaluation.
So
1. compute neighbourhood feature via aggregation
2. concat neighbourhood feature with local feature and previous output
3. perform a forward pass of a neural network.


In [64]:
def forward(result, neigh, feature, eval_logic, log_enable=False):
  if(log_enable):
    print("Input = ", result)
  reshape = tf.reshape(result[:,:, 1], feature.shape[0])
  neigh_evaluation = tf.reduce_sum(tf.multiply(neigh, reshape), 1)
  input_network = tf.concat([input[:,:, 0], neigh_evaluation[:, tf.newaxis]], 1)
  if(log_enable):
    print("Neighbour Aggregation = ", input_network)
  result = eval_logic(input_network[:, tf.newaxis])
  other = tf.concat([feature, result], axis = 1)
  if(log_enable):
    print("Output = ", result)
  return tf.expand_dims(other, [1])

# Evaluation function
The forward pass needs a function to compute the next output. In this case I use RNN combined with a Dense layer (i.e. a matrix multiplicaiton)

In [65]:
rnn0 = tf.keras.layers.LSTM(units = 10, activation='relu', return_sequences=False, stateful=True)
rnn = tf.keras.layers.Dense(units = 1, activation='relu')
model = tf.keras.Sequential([rnn0,rnn])
@tf.function
def rnn_pass(input):
  return model(input)

def forward_rnn(result, neigh, feature, log_enable=False):
  return forward(result, neigh, feature, rnn_pass, log_enable)

# Train function

In [66]:
## TODO pass input and network
def train(iteration, stabilise_in, stabilisation_check, loss, optimizer):
  for j in range(iteration):
    with tf.GradientTape() as tape:
      result = input
      to_backprop = 0
      for i in range(stabilise_in):
        result = forward_rnn(result, neigh, feature)
      for i in range(stabilisation_check):
        result = forward_rnn(result, neigh, feature)
        to_backprop += 1 / stalization_check * loss(ground, result[:, 0, :])
      rnn0.reset_states() ## todo improve
      rnn_g, rnn0_g = tape.gradient(to_backprop, [rnn.weights, rnn0.weights])
      optimizer.apply_gradients(zip(rnn_g, rnn.weights))
      optimizer.apply_gradients(zip(rnn0_g, rnn0.weights))
    print("Epoch ", j ,"Loss = ", tf.reduce_sum(to_backprop))  


## RNN With Aggregation


In [68]:
#seed(42)
#tf.random.set_seed(42)
iteration = 5000
stabilise_in = 3
stabilisation_check = 10

loss = tf.losses.mse
optimizer = tf.optimizers.Adam()
model.reset_states()
train(iteration, stabilise_in, stabilisation_check, loss, optimizer)

result = input
for i in range(20):
  result = forward_rnn(result, neigh, feature, True)
print(result)
rnn0.reset_states()

Epoch  0 Loss =  tf.Tensor(6.4471292, shape=(), dtype=float32)
Epoch  1 Loss =  tf.Tensor(6.4368777, shape=(), dtype=float32)
Epoch  2 Loss =  tf.Tensor(6.427545, shape=(), dtype=float32)
Epoch  3 Loss =  tf.Tensor(6.4181437, shape=(), dtype=float32)
Epoch  4 Loss =  tf.Tensor(6.407871, shape=(), dtype=float32)
Epoch  5 Loss =  tf.Tensor(6.397258, shape=(), dtype=float32)
Epoch  6 Loss =  tf.Tensor(6.386204, shape=(), dtype=float32)
Epoch  7 Loss =  tf.Tensor(6.37469, shape=(), dtype=float32)
Epoch  8 Loss =  tf.Tensor(6.3627076, shape=(), dtype=float32)
Epoch  9 Loss =  tf.Tensor(6.350253, shape=(), dtype=float32)
Epoch  10 Loss =  tf.Tensor(6.337322, shape=(), dtype=float32)
Epoch  11 Loss =  tf.Tensor(6.3239117, shape=(), dtype=float32)
Epoch  12 Loss =  tf.Tensor(6.3100176, shape=(), dtype=float32)
Epoch  13 Loss =  tf.Tensor(6.2956343, shape=(), dtype=float32)
Epoch  14 Loss =  tf.Tensor(6.2807617, shape=(), dtype=float32)
Epoch  15 Loss =  tf.Tensor(6.265396, shape=(), dtype=floa

KeyboardInterrupt: ignored

In [69]:
feature = tf.constant([
                     [1], 
                     [0],
                     [0],
                     [0],
                     [0]
                    ], dtype=tf.float32)
output = tf.constant([
                     [-1.0], 
                     [-1.0],
                     [-1.0],
                     [-1.0],
                     [-1.0]
                    ], dtype=tf.float32)

input = tf.concat([feature, output], axis = 1)
input = input[:,tf.newaxis, :]
neigh = tf.constant([
                     [0, 1, 0, 0, 0], 
                     [1, 0, 1, 0, 0],
                     [0, 1, 0, 1, 0],
                     [0, 0, 1, 0, 1],
                     [0, 0, 0, 1, 0],
                    ], dtype=tf.float32)

result = input
rnn0.reset_states()
for i in range(20):
  result = forward_rnn(result, neigh, feature, True)
print(result)

Input =  tf.Tensor(
[[[ 1. -1.]]

 [[ 0. -1.]]

 [[ 0. -1.]]

 [[ 0. -1.]]

 [[ 0. -1.]]], shape=(5, 1, 2), dtype=float32)
Neighbour Aggregation =  tf.Tensor(
[[ 1. -1.]
 [ 0. -2.]
 [ 0. -2.]
 [ 0. -2.]
 [ 0. -1.]], shape=(5, 2), dtype=float32)


ValueError: ignored